# Notebook de Generación de Versos
En este fichero enfocado para la entrega final del proyecto, experimentaremos con diferentes modelos de generación de texto. Donde nuestro objetivo va a ser conseguir nuevos versos, llamados `aleya`, que aporten contenido al Corán.

El enfoque que tomaremos será realizar comparaciones entre modelos generativos especializados en el árabe y modelos generativos generalistas (que sirvan para todas los idiomas).

Por otra parte, usaremos una **Large Language Model** (`LLM`) para evaluar la cohesión, coherencia y otras demás métricas de los versos generados. Dado el caso de que no haya ningún modelo evaluador árabe disponible, implementaremos otras *LLMs* manualmente para realizar esta segunda tarea. 

(zerbait gehiago gehitu/aldatu dezakegu)

## Dependencias

In [1]:
# Dependencias

## Generación de Aleyas

## Evaluación del Texto Generado